# Fidelity Training Script

This script trains the convolution neural network to recognize $\mathcal{F}$ values for every number of bases employed in compressive tomography, and later predicts the values with test examples. The complete simulated data consist of those from both statistically noiseless and noisy cases of ACT and random Haar schemes (4 batches). Each batch consists of $m=5000$ training datasets generated from a separate MATLAB script that simulates the ACT and random Haar schemes on 4-qubit systems ($d=2^4=16$).

### Loads packages.

In [ ]:
import scipy.io as spi
import os

import tensorflow as tf
import numpy as np
import pandas as pd
import seaborn as sns
import itertools
import json
import h5py

from keras.layers import Concatenate, Dense, Dropout, Conv2D, MaxPooling2D, Flatten, merge, Input, ZeroPadding2D, Activation, Add, AveragePooling2D
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras.models import Sequential, load_model, Model
from keras import optimizers, regularizers
from keras import backend as K
from keras.layers.normalization import BatchNormalization

from sklearn import preprocessing
from sklearn.model_selection import train_test_split

### Defines work parameters

In [ ]:
d=16 # Hilbert-space dimension
m=8000 # Number of training datasets
num_classes=1 # Set to 1 by default since we are fitting a single continuous output value

# Defines the working directory containing all generated MATLAB MAT training data files
work_dir='../training/'+str(m)+'_perf_noisy_ex/'

# Defines ICCNet model storage directory
stor_dir=work_dir+'FidNet_trained_files/'

### Training commences

In [ ]:
# Loops over different number of bases "Kbas" for training FidNet

for l in np.linspace(1,10,10): # Defines the number of bases "Kbas" for training the ConvNet.

    Kbas=int(l)

    # Loads the relevant input matrix for the defined Kbas.
    X = spi.loadmat(work_dir+'X_'+str(Kbas)+'.mat')['X']

    # Loads the fidelity values.
    y = spi.loadmat(work_dir+'y_'+str(Kbas)+'.mat')['y']
    y = y[:,1].reshape([m,1])
    y = np.abs(y)

    if os.path.isdir(stor_dir)==False:
        os.mkdir(stor_dir)

    # --------------------------------------------------------------------------
    # Data preprocessing and splitting to 'train', 'validation' and 'test' sets.
    # --------------------------------------------------------------------------  

    # Splits the full dataset into training, validation and test subsets.
    x_train0, x_rest, y_train, y_rest = train_test_split(X, y, test_size=0.2, shuffle=True)
    x_test, x_val, y_test, y_val = train_test_split(x_rest, y_rest, test_size=0.5, shuffle=True)

    # Performs standard data normalization and scaling.
    x_train = preprocessing.StandardScaler().fit(x_train0).transform(x_train0)
    x_train = x_train.astype('float32')
    x_test = preprocessing.StandardScaler().fit(x_train0).transform(x_test)
    x_test = x_test.astype('float32')
    x_val = preprocessing.StandardScaler().fit(x_train0).transform(x_val)
    x_val = x_val.astype('float32')  

    # ---------------------------------------------
    # FidNet model definition and parametrization.
    # ---------------------------------------------

    # Defines gradient optimization batch size and training epochs.
    batch_size = 1024
    epochs = 500
    
    tf.keras.initializers.he_normal()

    # Converts inputs to single-channel 2D images.
    img_rows = int(np.ceil(np.power(x_train.shape[1],0.5)))
    img_cols = img_rows
    x_train = np.concatenate((x_train,np.zeros((x_train.shape[0],img_rows*img_cols-x_train.shape[1]))),axis=1)
    x_test = np.concatenate((x_test,np.zeros((x_test.shape[0],img_rows*img_cols-x_test.shape[1]))),axis=1)
    x_val = np.concatenate((x_val,np.zeros((x_val.shape[0],img_rows*img_cols-x_val.shape[1]))),axis=1)

    K.clear_session()
    x_train=x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test=x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    x_val=x_val.reshape(x_val.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1) 
    
    # Sets up neural network
    X_input = Input(input_shape) 

    Xnn = Conv2D(4, kernel_size = (5,5), padding='valid',strides=1, kernel_initializer = 'he_uniform')(X_input)
    Xnn = BatchNormalization()(Xnn)
    Xnn = Activation('relu')(Xnn)
    Xnn = Dropout(0.3)(Xnn)

    Xnn = Conv2D(8, kernel_size = (5,5), padding='valid',strides=1, kernel_initializer = 'he_uniform')(Xnn)
    Xnn = BatchNormalization()(Xnn)
    Xnn = Activation('relu')(Xnn)
    Xnn = Dropout(0.5)(Xnn)

    Xnn = Conv2D(16, kernel_size = (5,5), padding='valid',strides=1, kernel_initializer = 'he_uniform')(Xnn)
    Xnn = BatchNormalization()(Xnn)
    Xnn = Activation('relu')(Xnn)
    Xnn = Dropout(0.5)(Xnn)

    Xnn = AveragePooling2D(pool_size=(2,2),strides=2)(Xnn)

    Xnn = Flatten()(Xnn)

    Xnn = Dense(32)(Xnn)
    Xnn = BatchNormalization()(Xnn)
    Xnn = Activation('sigmoid')(Xnn)
    Xnn = Dropout(0.5)(Xnn)

    Xnn = Dense(num_classes,activation='sigmoid')(Xnn)

    model = Model(inputs = X_input, outputs = Xnn)

    model.summary()

    model.compile(loss='mse',optimizer='nadam')

    # -----------------
    # Network training.
    # -----------------

    # Sets up the scoreboard and checkpoint monitor to record training results.    
    checkpointer = ModelCheckpoint(filepath=stor_dir+"model_FidNet_K_"+str(Kbas)+".h5",
                                   verbose=0, monitor='val_loss', mode='min', save_best_only=True)
    tensorboard = TensorBoard(log_dir='./logs',
                              histogram_freq=0,
                              write_graph=True,
                              write_images=True)

    # Trains ICCNet with training and validation data subsets and records all results.
    history = model.fit(x_train, y_train,
                        epochs=epochs,
                        batch_size=batch_size,
                        validation_data=(x_val, y_val),
                        verbose=1,
                        callbacks=[checkpointer, tensorboard]).history

    with open(stor_dir+"model_FidNet_K_"+str(Kbas)+"_hist.txt", 'w') as outfile:  
        json.dump(history, outfile)

    # Evaluates the trained model with test data subsets.
    score=model.evaluate(x_test,y_test,verbose=0)

    print('Test loss:',score)

    # Stores all split data subsets into a dictionary and saves it into a MATLAB cell file.
    Xy_dict = {'x_val': x_val, 'y_val': y_val, 'x_test': x_test, 'y_test': y_test, 'x_train0': x_train0, 'y_train': y_train}
    spi.savemat(stor_dir+"model_FidNet_K_"+str(Kbas)+"_Xy.mat", Xy_dict)